In [1]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install --upgrade pip

In [3]:
import psycopg2
import pandas as pd
import csv
import os
from pathlib import Path

## Extraction

In [4]:
def extract(folder_path):
    """
    Load all CSV files in the given folder into dataFrames.
    
    Args:
        folder_path (str or Path): Path to the folder containing CSV files.

    Returns:
        dict: A dictionary with file names as keys and DataFrames as values.
    """
    data_folder = Path(folder_path)
    csv_files = list(data_folder.glob("*.csv"))
    dfs = {}

    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            key = file_path.stem.replace(" ","_").lower()
            dfs[key] = df
            dataframes[file_path.name] = df
            print(f"Loaded: {file_path.name} dfs['{key}'], {df.shape[0]} rows {df.shape[1]} cols")
        except pd.errors.EmptyDataError:
            print(f"Skipped (empty): {file_path.name}")
        except pd.errors.ParserError:
            print(f"Skipped (parser error): {file_path.name}")
        except Exception as e:
            print(f"Failed to load {file_path.name}: {e}")
    return dfs


In [5]:
df = extract('data')

Failed to load brands.csv: name 'dataframes' is not defined
Failed to load categories.csv: name 'dataframes' is not defined
Failed to load customers.csv: name 'dataframes' is not defined
Failed to load orders.csv: name 'dataframes' is not defined
Failed to load order_items.csv: name 'dataframes' is not defined
Failed to load products.csv: name 'dataframes' is not defined
Failed to load staffs.csv: name 'dataframes' is not defined
Failed to load stocks.csv: name 'dataframes' is not defined
Failed to load stores.csv: name 'dataframes' is not defined


## Transform

In [6]:
df['customers'].head()

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


In [7]:
import hashlib

In [8]:
customers = df["customers"]
customers.head()

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


In [9]:
# remove PII
#assumptions are that the null phone numbers will be updated at some point
def hash_value(val):
    if pd.isna(val):
        return ""
    return hashlib.sha256(str(val).encode("utf-8")).hexdigest()

df["customers"]["phone"] = df["customers"]["phone"].apply(hash_value)
df["customers"]["email"] = df["customers"]["email"].apply(hash_value)
df["customers"].head()

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,,80d3fb98b10fa0cb880d7ea7ce6e673a3740e3c15138af...,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,,14963ce86476128ab360ceb6ff80a1b377a0aa531d204c...,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,,cd85931dec22b5276e6ca71d9e2d5f6adc99cda2992a6c...,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,,7e724a8da0c75960dc02b4affd782c570921403b17fbdb...,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,3d73364c47e5bcef4aa230e9d31b8d1bc10d4ae5b1b0c2...,0963fe709fbfbb928cb8a0de6132b82454215188c6131c...,107 River Dr.,Sacramento,CA,95820


In [10]:
df["staffs"].head()

,staff_id,first_name,last_name,email,phone,active,store_id,manager_id
0,1,Fabiola,Jackson,fabiola.jackson@bikes.shop,(831) 555-5554,1,1,NaN
1,2,Mireya,Copeland,mireya.copeland@bikes.shop,(831) 555-5555,1,1,1.0
2,3,Genna,Serrano,genna.serrano@bikes.shop,(831) 555-5556,1,1,2.0
3,4,Virgie,Wiggins,virgie.wiggins@bikes.shop,(831) 555-5557,1,1,2.0
4,5,Jannette,David,jannette.david@bikes.shop,(516) 379-4444,1,2,1.0


In [11]:
df["staffs"]["phone"] = df["staffs"]["phone"].apply(hash_value)
df["staffs"]["email"] = df["staffs"]["email"].apply(hash_value)

## Loading

In [12]:
pip install dotenv

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [13]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text


In [14]:
def create_database():
    # Load .env variables
    load_dotenv()

    # Fetch DB credentials
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT", "5432")
    db_name = os.getenv("DB_NAME")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    # Create database connection to default DB
    conn = psycopg2.connect(f"""host={db_host} 
                                dbname={db_name}
                                user={db_user} 
                                password={db_password}
                             """)
    # Autocommit
    conn.set_session(autocommit=True)
    
    # Create cursor object
    cur = conn.cursor()

    # Create bike_store database
    cur.execute("DROP DATABASE IF EXISTS Bike_Store")
    cur.execute("CREATE DATABASE Bike_Store")

    # Close default db connection
    cur.close()
    conn.close()

    # Connect to Bike_Store database
    conn = psycopg2.connect(f"host={db_host} dbname=bike_store user={db_user} password={db_password}")
    cur = conn.cursor()

    return cur, conn

In [15]:
cur, conn = create_database()

In [16]:
cur.execute("SELECT current_database();")
print(cur.fetchone())

('bike_store',)


In [17]:

tables = {
    ("customer_table","""CREATE TABLE IF NOT EXISTS Customers(customer_id INT PRIMARY KEY, 
                  first_name VARCHAR (255) NOT NULL, 
                  last_name VARCHAR (255) NOT NULL, 
                  street VARCHAR (255),
                  city VARCHAR (50),
                  state VARCHAR (25),
                  zip_code VARCHAR (5)
                  );
                  """),
                  
    ("orders","""CREATE TABLE IF NOT EXISTS Orders(
                order_id INT PRIMARY KEY,
	            customer_id INT,
	            order_status INT NOT NULL,
	            -- Order status: 1 = Pending; 2 = Processing; 3 = Rejected; 4 = Completed
	            order_date DATE NOT NULL,
	            required_date DATE NOT NULL,
	            shipped_date DATE,
	            store_id INT NOT NULL,
	            staff_id INT NOT NULL,
	            FOREIGN KEY (customer_id) REFERENCES customers (customer_id) ON DELETE CASCADE ON UPDATE CASCADE,
	            FOREIGN KEY (store_id) REFERENCES stores (store_id) ON DELETE CASCADE ON UPDATE CASCADE,
	            FOREIGN KEY (staff_id) REFERENCES staff (staff_id) ON DELETE NO ACTION ON UPDATE NO ACTION
                );
                """),

    ("order_items","""CREATE TABLE IF NOT EXISTS Order_items(
                    order_id INT,
	                item_id INT,
	                product_id INT NOT NULL,
	                quantity INT NOT NULL,
	                list_price DECIMAL (10, 2) NOT NULL,
	                discount DECIMAL (4, 2) NOT NULL DEFAULT 0,
	                PRIMARY KEY (order_id, item_id),
	                FOREIGN KEY (order_id) REFERENCES orders (order_id) ON DELETE CASCADE ON UPDATE CASCADE,
	                FOREIGN KEY (product_id) REFERENCES products (product_id) ON DELETE CASCADE ON UPDATE CASCADE
                    );
                    """),

    ("stores","""CREATE TABLE IF NOT EXISTS Stores(
              store_id INT PRIMARY KEY,
              store_name VARCHAR (255) NOT NULL,
              street VARCHAR (255),
              city VARCHAR (255),
              state VARCHAR (10),
              zip_code VARCHAR (5)
              );
            """),

    ("staffs","""CREATE TABLE IF NOT EXISTS Staff(
               staff_id INT PRIMARY KEY,
               first_name VARCHAR (50) NOT NULL,
               last_name VARCHAR (50) NOT NULL,
               active INT NOT NULL,
               store_id INT NOT NULL,
               manager_id INT,
               FOREIGN KEY (store_id) REFERENCES stores (store_id) ON DELETE CASCADE ON UPDATE CASCADE,
               FOREIGN KEY (manager_id) REFERENCES staff(staff_id) ON DELETE NO ACTION ON UPDATE NO ACTION
               );"""),

    ("categories","""CREATE TABLE IF NOT EXISTS Categories(category_id INT PRIMARY KEY,
                 category_name VARCHAR (255) NOT NULL
                 );"""),

    ("brands","""CREATE TABLE IF NOT EXISTS Brands(brand_id INT PRIMARY KEY,
                brand_name VARCHAR (255) NOT NULL);"""),

    ("products","""CREATE TABLE IF NOT EXISTS Products(product_id INT PRIMARY KEY,
                 product_name VARCHAR (255) NOT NULL,
                 brand_id INT NOT NULL,
                 category_id INT NOT NULL,
                 model_year SMALLINT NOT NULL,
                 list_price DECIMAL (10, 2) NOT NULL,
                 FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE ON UPDATE CASCADE,
                 FOREIGN KEY (brand_id) REFERENCES brands(brand_id) ON DELETE CASCADE ON UPDATE CASCADE
                 );"""),

    ("stocks","""CREATE TABLE IF NOT EXISTS Stocks(store_id INT,
                product_id INT,
                quantity INT,
                PRIMARY KEY (store_id, product_id),
                FOREIGN KEY (store_id) REFERENCES stores (store_id) ON DELETE CASCADE ON UPDATE CASCADE,
                FOREIGN KEY (product_id) REFERENCES products (product_id) ON DELETE CASCADE ON UPDATE CASCADE)""")

                  }

In [18]:
for name, sql in tables:
    try:
        cur.execute(sql)
        conn.commit()
        print(f"Created table: {name}")
    except Exception as e:
        print(f"Failed to create {name}: {e}")
        conn.rollback()

Created table: customer_table
Failed to create stocks: relation "stores" does not exist

Failed to create products: relation "categories" does not exist

Created table: categories
Failed to create staffs: relation "stores" does not exist

Created table: stores
Created table: brands
Failed to create orders: relation "staff" does not exist

Failed to create order_items: relation "orders" does not exist



In [19]:
from sqlalchemy import create_engine

In [20]:
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT", "5432")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [21]:
connection_uri = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [22]:
engine = create_engine(connection_uri)

In [23]:
df.keys()

dict_keys(['brands', 'categories', 'customers', 'orders', 'order_items', 'products', 'staffs', 'stocks', 'stores'])

In [24]:
for table, dfs in df.items():
    try:
        dfs.to_sql(table, engine, if_exists="replace", index=False)
        print(f" Loaded {len(dfs)} rows into '{table}'")
    except Exception as e:
        print(f" Failed to load '{table}': {e}")


 Loaded 9 rows into 'brands'
 Loaded 7 rows into 'categories'
 Loaded 1445 rows into 'customers'
 Loaded 1615 rows into 'orders'
 Loaded 4722 rows into 'order_items'
 Loaded 321 rows into 'products'
 Loaded 10 rows into 'staffs'
 Loaded 939 rows into 'stocks'
 Loaded 3 rows into 'stores'


In [25]:
conn.close()
cur.close()